In [45]:
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold,StratifiedKFold
from rdkit import Chem
from rdkit.Chem import MACCSkeys
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
import random
import re
import torch

In [36]:
def fetch(target):
    fname_human = f'{target}_HUMAN.csv'
    df_human = pd.read_csv(fname_human, sep=';')

    fname_rat = f'{target}_RAT.csv'
    df_rat = pd.read_csv(fname_rat, sep=';')

    fname_mouse = f'{target}_MOUSE.csv'
    df_mouse = pd.read_csv(fname_mouse, sep=';')

    df = pd.concat([df_human, df_rat, df_mouse])

    mask = df['Standard Type'] == 'IC50'

    td = 10
    pmask = (df[mask]['Standard Value'] < td) & (df[mask]['Standard Value'] > 0)
    nmask = df[mask]['Standard Value'] > td

    len_p = sum(pmask)
    len_n = sum(nmask)

    print(f'靶点：{target}, 正样本数量：', len_p)
    print(f'靶点：{target}, 负样本数量：', len_n)

    df_p = df[mask][pmask]
    df_n = df[mask][nmask]

    return df_p, df_n, len_p

In [37]:
df_ep4_p, df_ep4_n, len_ep4_p = fetch('EP4')

靶点：EP4, 正样本数量： 161
靶点：EP4, 负样本数量： 482


In [ ]:
n, bins, patches = plt.hist(df_ep4_p['Standard Value'], 50, density=False)
plt.show()

In [42]:
outputs = []
data_mix = []

for i in range(len_ep4_p):
    index = df_ep4_p.index[i]
    smiles = df_ep4_p['Smiles'][index]
    ic50 = df_ep4_p['Standard Value'][index]

    try:
        m=Chem.MolFromSmiles(smiles)
        # print(smiles)
    except Exception as e:              # 不能解析的话跳过
        continue

    desc = 'It can bind to Prostanoid EP4 receptor.'
    outputs.append(['None', 'None', desc, smiles])
    data_mix.append([smiles,ic50,1])

for i in range(len_ep4_p):
    index = df_ep4_n.index[i]
    smiles = df_ep4_n['Smiles'][index]
    ic50 = df_ep4_n['Standard Value'][index]

    try:
        m=Chem.MolFromSmiles(smiles)
        # print(smiles)
    except Exception as e:              # 不能解析的话跳过
        continue

    desc = 'It can not bind to Prostanoid EP4 receptor.'
    outputs.append(['None', 'None', desc, smiles])
    data_mix.append([smiles,ic50,0])

len(outputs)

314

In [35]:
outputs = pd.DataFrame(data=outputs, columns=['cid','iupac','desc','smiles'])
outputs.to_csv(f'../data/train/EP4.csv', index=False)
print('Saved.')

Saved.


# 分类器

In [43]:
df_mix = pd.DataFrame(data=data_mix, columns=['smiles','IC50','label'])
df_mix = df_mix.sample(frac=1)
df_mix

,smiles,IC50,label
110,Cc1ccc(-c2cccc(CO)c2)nc1C(=O)Nc1c(C)ccc(C(=O)O...,2.36,1
97,Cc1ccc(C(=O)O)c(C)c1NC(=O)c1cc(-c2ccccc2)cc2cc...,5.65,1
88,C[C@H](NC(=O)c1cc(Cl)cc2ccn(Cc3cccc(Cl)c3)c12)...,1.50,1
144,C[C@H](NC(=O)c1c(Cc2ccc(OC(F)(F)F)c(Cl)c2)sc2c...,6.60,1
107,C[C@H](NC(=O)c1c(C(F)F)nn(C)c1Oc1cccc(C(F)(F)F...,7.50,1
...,...,...,...
172,O=C(O)Cn1c2c(c3cc(C(F)(F)F)ccc31)CN(C(=O)c1ccc...,10000.00,0
291,CCCCC[C@H](O)/C=C/[C@@H]1[C@@H](C/C=C\CCCC(=O)...,400.00,0
132,C[C@@H](CCCc1ccccc1)[C@H](O)/C=C/[C@H]1CC(F)(F...,0.74,1
209,Cc1oc(C(=O)O)cc1COc1cccc(-c2ccccc2)c1,10000.00,0


In [44]:
smiles_list = df_mix['smiles'].values
y = df_mix['label'].values
ic50s = df_mix['IC50'].values
sum(y==0)

153

In [49]:
x = []
for smiles in smiles_list:
    mol=Chem.MolFromSmiles(smiles)
    fp = MACCSkeys.GenMACCSKeys(mol)
    # fp_bits = fp.ToBitString()
    fp_bits = fp.ToList()
    x.append(fp_bits)
x = np.array(x)
x.shape

(314, 167)

In [50]:
sfolder = StratifiedKFold(n_splits=10,random_state=0,shuffle=True)

In [52]:
f1_list = []
precision_list =  []
regr_list = []
for train, test in sfolder.split(x,y,ic50s):
    # print(sum(y[test]==0))
    # print(sum(y[test]==1))
    regr = make_pipeline(SVC(kernel='rbf', probability = True))
    regr.fit(x[train], y[train])
    pred = regr.predict(x[test])

    # for i in range(len(pred)):
    #     if pred[i] != y[test][i]:
    #         print(f"预测值: {pred[i]}, 实际值: {y[test][i]}, IC50: {ic50s[test][i]}")

    # print(pred)
    # print(y[test])
    f1 = f1_score(y[test].reshape(-1,1), pred.reshape(-1,1), average='micro')
    p_score = precision_score(y[test], pred, average='micro')
    f1_list.append(f1)
    precision_list.append(p_score)
    print(f"f1: {f1}; precision: {p_score}.")

    regr_list.append(regr)

f1_ts = torch.Tensor(f1_list)
precision_ts = torch.Tensor(precision_list)

print(f"F1 mean: {f1_ts.mean()}, std: {f1_ts.std()}.")
print(f"Precision mean: {precision_ts.mean()}, std: {precision_ts.std()}.")

f1: 0.84375; precision: 0.84375.
f1: 0.6875; precision: 0.6875.
f1: 0.71875; precision: 0.71875.
f1: 0.65625; precision: 0.65625.
f1: 0.7741935483870968; precision: 0.7741935483870968.
f1: 0.7096774193548389; precision: 0.7096774193548387.
f1: 0.7096774193548389; precision: 0.7096774193548387.
f1: 0.9032258064516129; precision: 0.9032258064516129.
f1: 0.7741935483870968; precision: 0.7741935483870968.
f1: 0.8709677419354839; precision: 0.8709677419354839.
F1 mean: 0.7648184895515442, std: 0.08353233337402344.
Precision mean: 0.7648184895515442, std: 0.08353233337402344.


In [53]:
import joblib
joblib.dump(regr_list[7], "SVC_EP4.m")

['SVC_EP4.m']